In [1]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
### HAMILTONIAN start
Molecule = 'LiH'
geometry = None # [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
basis = 'sto-6g'


### Get Hamiltonian
Hamilt = Hamiltonian(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='BK')
### HAMILTONIAN end

#####################################


In [2]:
from quchem.Ansatz_Generator_Functions import *

##
NOON_spins_combined, NMO_basis = Hamilt.Get_NOON()
##
Hamilt.Get_CCSD_Amplitudes()



ansatz_obj = Ansatz(Hamilt.molecule.n_electrons, Hamilt.molecule.n_qubits)
reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab, reduced_theta_parameters_ia, reduced_theta_parameters_ijab =ansatz_obj.Remove_NOON_terms(
    NOON=NOON_spins_combined,
    occ_threshold= 1.999,
    unocc_threshold=1e-4,
    indices_to_remove_list_manual=None, 
    single_cc_amplitudes=Hamilt.molecule.single_cc_amplitudes,
    double_cc_amplitudes=Hamilt.molecule.double_cc_amplitudes,
    singles_hamiltonian=Hamilt.singles_hamiltonian,
    doubles_hamiltonian=Hamilt.doubles_hamiltonian,
    tol_filter_small_terms=None)
reduced_Sec_Quant_CC_ops_ijab

[-1.0 [2^ 3^ 4 5] +
 1.0 [5^ 4^ 3 2],
 -1.0 [2^ 3^ 4 7] +
 1.0 [7^ 4^ 3 2],
 -1.0 [2^ 3^ 4 9] +
 1.0 [9^ 4^ 3 2],
 -1.0 [2^ 3^ 6 7] +
 1.0 [7^ 6^ 3 2],
 -1.0 [2^ 3^ 6 9] +
 1.0 [9^ 6^ 3 2],
 -1.0 [2^ 3^ 8 9] +
 1.0 [9^ 8^ 3 2]]

In [6]:
NOON_spins_combined

array([1.99991509e+00, 1.96744892e+00, 2.71958861e-02, 2.68032899e-03,
       2.68032899e-03, 7.94453898e-05])

In [5]:
ia_terms, ijab_terms, ia_theta, ijab_theta = ansatz_obj.Get_ia_and_ijab_terms()

print('REDUCTION')
print('ia_terms', len(ia_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ia))
print('ijab_terms', len(ijab_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ijab))
reduced_Sec_Quant_CC_ops_ijab

REDUCTION
ia_terms 16 TO 6
ijab_terms 42 TO 6


[-1.0 [2^ 3^ 4 5] +
 1.0 [5^ 4^ 3 2],
 -1.0 [2^ 3^ 4 7] +
 1.0 [7^ 4^ 3 2],
 -1.0 [2^ 3^ 4 9] +
 1.0 [9^ 4^ 3 2],
 -1.0 [2^ 3^ 6 7] +
 1.0 [7^ 6^ 3 2],
 -1.0 [2^ 3^ 6 9] +
 1.0 [9^ 6^ 3 2],
 -1.0 [2^ 3^ 8 9] +
 1.0 [9^ 8^ 3 2]]

In [4]:
# note transform set to 'BK'
Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab = ansatz_obj.UCCSD_single_trotter_step(reduced_Sec_Quant_CC_ops_ia, 
                                                                                                                reduced_Sec_Quant_CC_ops_ijab,
                                                                                                                   transformation='BK')

LOOK at:

PHYS. REV. X, **8**, 031022 (2018)

In [ ]:
new_BK = BK_Qubit_Reordering(
                QubitHamiltonian,
                 Qubit_Op_list_Second_Quant_CC_Ops_ia,
                 Qubit_Op_list_Second_Quant_CC_Ops_ijab,
                 Hamilt.molecule.n_qubits,
                ansatz_obj.Get_BK_HF_state_in_OCC_basis(),
                Hamilt.molecule.n_electrons)

REDUCED_Qubit_MolecularHamiltonian = new_BK.Get_Reordered_Hamiltonian_2_qubits_removed()
REDUCED_Qubit_MolecularHamiltonian

In [ ]:
Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(REDUCED_Qubit_MolecularHamiltonian)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)
anti_commuting_sets

In [ ]:
Qubit_Op_list_Second_Quant_CC_Ops_ijab

In [ ]:
new_BK.Get_Reordered_CC_qubit_terms_2_qubits_removed()

In [ ]:
from quchem.Simulating_Quantum_Circuit import *

In [ ]:
n_shots= 1000


def VQE_experiment_ENERGY(theta_ia_ijab_combined_list):
    theta_params_ia = [theta_ia_ijab_combined_list[0], theta_ia_ijab_combined_list[1]]
    theta_params_ijab = [theta_ia_ijab_combined_list[2]]
    
    
    ansatz_cirq_circuit = full_ansatz_Q_Circ.Get_Full_HF_UCCSD_QC(theta_params_ia, 
                                                              theta_params_ijab)
    

    VQE_exp = VQE_Experiment(QubitHamiltonian, ansatz_cirq_circuit, n_shots)
    return VQE_exp.Calc_Energy().real

In [ ]:
theta_combined = [1,2, np.pi]

VQE_experiment_ENERGY(theta_combined)

In [ ]:
import random

theta_ia_random_input = [random.uniform(0, 2*np.pi) for _ in range(len(Sec_Quant_CC_ops_ia))]
theta_ijab_random_input = [random.uniform(0, 2*np.pi) for _ in range(len(Sec_Quant_CC_ops_ijab))]

theta_combined_random_input = [*theta_ia_random_input, *theta_ijab_random_input]

In [ ]:
### optimizer
from quchem.Scipy_Optimizer import *

GG = Optimizer(VQE_experiment_ENERGY, theta_combined_random_input, 'Nelder-Mead', store_values=True, display_iter_steps=True,
               tol=1e-5,
               display_convergence_message=True)
GG.get_env(50)
GG.plot_convergence()
plt.show()

In [ ]:
Hamilt.molecule.fci_energy

In [ ]:
from quchem.TensorFlow_Opt import *

**gradient is given by**

https://arxiv.org/pdf/1906.08728.pdf

$$\frac{\partial O(\theta)}{\partial \theta}=\left\langle\overrightarrow{0}\left|\hat{U}^{\dagger} \hat{R}_{y}^{C \dagger}(\theta+\pi / 4) \hat{V}^{\dagger} \hat{O} \hat{V} \hat{R}_{y}^{C}(\theta+\pi / 4) \hat{U}\right| \overrightarrow{0}\right\rangle -\left\langle\overrightarrow{0}\left|\hat{U}^{\dagger} \hat{R}_{y}^{C \dagger}(\theta-\pi / 4) \hat{V}^{\dagger} \hat{O} \hat{V} \hat{R}_{y}^{C}(\theta-\pi / 4) \hat{U}\right| \overrightarrow{0}\right\rangle$$

$$\frac{\partial O(\theta)}{\partial \theta} =O(\theta+\pi / 4)-O(\theta-\pi / 4)$$

In [ ]:
def calc_gradient(theta_ia_theta_jab_list):
    
    grad_list=[]
    
    for index, theta in enumerate(theta_ia_theta_jab_list):
        
        new_theta_list = theta_ia_theta_jab_list.copy()
        new_theta_list[index] = theta + np.pi/4
        
        Obs_PLUS = VQE_experiment_ENERGY(new_theta_list)
               
        
        new_theta_list[index] = theta - np.pi/4
        Obs_MINUS = VQE_experiment_ENERGY(new_theta_list)
        
        gradient = Obs_PLUS - Obs_MINUS
        
        grad_list.append((gradient, theta))
        

    return grad_list

note:
this is very SLOW as it has to run a separate experiment TWICE for each parameter before taking a step!

In [ ]:
X0 = [random.uniform(0, 2*np.pi) for _ in range(len(Sec_Quant_CC_ops_ia) + len(Sec_Quant_CC_ops_ijab))]

tf_opt = Tensor_Flow_Optimizer(VQE_experiment_ENERGY, X0, 'Adam', calc_gradient, learning_rate=0.1, beta1=0.9,
                            beta2=0.999, store_values=True, display_iter_steps=True)
tf_opt.optimize(50)
tf_opt.plot_convergence()

In [ ]:
from quchem.Adam_Optimizer import *

In [ ]:
def calc_gradient_ADAM(theta_ia_theta_jab_list):
    
    grad_list=[]
    
    for index, theta in enumerate(theta_ia_theta_jab_list):
        
        new_theta_list = theta_ia_theta_jab_list.copy()
        new_theta_list[index] = theta + np.pi/4
        
        Obs_PLUS = VQE_experiment_ENERGY(new_theta_list)
               
        
        new_theta_list[index] = theta - np.pi/4
        Obs_MINUS = VQE_experiment_ENERGY(new_theta_list)
        
        gradient = Obs_PLUS - Obs_MINUS
        
        grad_list.append(gradient)
        

    return np.array(grad_list)

In [ ]:
X0 = np.array([random.uniform(0, 2*np.pi) for _ in range(len(Sec_Quant_CC_ops_ia) + len(Sec_Quant_CC_ops_ijab))])

opt_params, list_of_inputs, list_of_outputs = Adam_Opt(X0, VQE_experiment_ENERGY,
                                                       calc_gradient_ADAM, 
                                                       learning_rate=0.1, 
                                                       beta_1=0.9, 
                                                       beta_2=0.999, 
                                                       epsilon=1e-8,
                                                       max_iter=50,
                                                       disp=True,
                                                       tolerance=1e-3,
                                                       store_steps=True)

VQE_experiment_ENERGY(opt_params)

In [ ]:
import matplotlib.pyplot as plt
# % matplotlib inline
plt.figure()
plt.plot(list_of_outputs)
plt.xlabel('iterations')
plt.ylabel('objective function value')
plt.show()